In [7]:
import datetime

import ee

ee.Initialize()

In [8]:
def date_range(start_dt, end_dt, days=1, skip_leap_days=True):
    """Generate dates within a range (inclusive)

    Parameters
    ----------
    start_dt : datetime
        start date.
    end_dt : datetime
        end date.
    days : int, optional
        Step size (the default is 1).
    skip_leap_days : bool, optional
        If True, skip leap days while incrementing (the default is True).

    Yields
    ------
    datetime

    """
    import copy
    curr_dt = copy.copy(start_dt)
    while curr_dt <= end_dt:
        if not skip_leap_days or curr_dt.month != 2 or curr_dt.day != 29:
            yield curr_dt
        curr_dt += datetime.timedelta(days=days)

In [25]:
start_dt = datetime.datetime(2017, 7, 15)
end_dt = datetime.datetime(2017, 7, 17)
overwrite_flag = False

for export_dt in date_range(start_dt, end_dt, days=1, skip_leap_days=True):
    print(export_dt.strftime('%Y-%m-%d'))
    image_id = '{}'.format(export_dt.strftime('%Y%j'))
    
    asset_id = 'projects/disalexi/alexi/CONUS_V001_mod/{}'.format(image_id)
    export_id = asset_id.replace('/', '_')
    
    input_img = ee.Image('projects/disalexi/alexi/CONUS_V001/{}'.format(image_id))
    export_img = input_img.multiply(1.0).float().rename(['et'])\
        .set({
            'system:time_start': input_img.get('system:time_start'),
            'system:index': input_img.get('system:index'),
            'ALEXI_VERSION': input_img.get('ALEXI_VERSION'),
            'DATE_INGESTED': input_img.get('DATE_INGESTED'),
            'DATE_UPDATED': datetime.datetime.today().strftime('%Y-%m-%d'),
            'ET_UNITS': 'MJ m-2 d-1',
        })
    # print(export_img.getInfo())

    export_crs = 'EPSG:4326'
    export_geo = '[0.04, 0.0, -125.03, 0.0, -0.04, 49.79]'
    export_shape = '1456x625'
    # export_geo = '[0.04, 0.0, -125.04, 0.0, -0.04, 49.8]'
    # export_geo = '[' + ','.join(list(map(str, export_geo))) + ']'
    # export_shape = '{0}x{1}'.format(*export_shape)

    try:
        asset_info = ee.Image(asset_id).getInfo()
        if overwrite_flag:
            print('  Asset already exists, removing')
            ee.data.deleteAsset(asset_id)
        else:
            print('  Asset already exists, skipping')
            continue
    except:
        pass
    
    task = ee.batch.Export.image.toAsset(
        image=export_img,
        description=export_id,
        assetId=asset_id,
        crs=export_crs,
        crsTransform=export_geo,
        dimensions=export_shape,
    )
    
    print('  Starting export task')
    task.start()
        
    # break

2017-07-15
{'type': 'Image', 'bands': [{'id': 'et', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1456, 625], 'crs': 'EPSG:4326', 'crs_transform': [0.04, 0.0, -125.04, 0.0, -0.04, 49.8]}], 'properties': {'ET_UNITS': 'MJ m-2 d-1', 'system:time_start': 1500076800000.0, 'DATE_UPDATED': '2019-04-30', 'ALEXI_VERSION': 1.0, 'system:index': '2017196', 'DATE_INGESTED': '2018-12-06'}}
